In [ ]:
import keyring
import getpass
import functools
import itertools
import tempfile
import io

import numpy as np
import matplotlib.pyplot as plt

import imageio
import skimage.transform

import pydicom

import segments

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys._experimental.autosegmentation import pipeline, mask

In [ ]:
segments_api_key = keyring.get_password('segments-ai', 'api-key')

if not segments_api_key:
    segments_api_key = getpass.getpass()
    keyring.set_password('segments-ai', 'api-key', segments_api_key)

In [ ]:
client = segments.SegmentsClient(segments_api_key)
dataset_name = 'SimonBiggs/AnimalContours' # Name of a dataset you've created on Segments.ai

In [ ]:
dataset = client.get_dataset(dataset_name)

In [ ]:
contouring_task = [item for item in dataset['tasks'] if item['name'] == 'contouring'][0]
categories = contouring_task['attributes']['categories']
category_id_map = {
    item['name']: item['id']
    for item in categories
}

category_id_map

In [ ]:
samples = client.get_samples(dataset_name)

ct_uid_to_upload_uuid = {
    item['name'].replace(".png", ""): item['uuid']
    for item in samples
}

In [ ]:
ct_uid = list(ct_uid_to_upload_uuid.keys())[0]

In [ ]:
ct_uid

In [ ]:
# ct_uid = '1.2.840.113704.1.111.3096.1537312918.112198'

In [ ]:
(
    data_path_root,
    structure_set_paths,
    ct_image_paths,
    ct_uid_to_structure_uid,
    structure_uid_to_ct_uids,
    names_map,
    structure_names_by_ct_uid,
    structure_names_by_structure_set_uid,
    uid_to_url,
    hash_path,
) = pipeline.get_dataset_metadata()

In [ ]:
# uid_to_url

In [ ]:
@functools.lru_cache()
def get_dcm_ct_from_uid(ct_uid):
    ct_path = ct_image_paths[ct_uid]
    dcm_ct = pydicom.read_file(ct_path, force=True)

    dcm_ct.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian

    return dcm_ct

@functools.lru_cache()
def get_dcm_structure_from_uid(structure_set_uid):
    structure_set_path = structure_set_paths[structure_set_uid]

    dcm_structure = pydicom.read_file(
        structure_set_path,
        force=True,
        specific_tags=["ROIContourSequence", "StructureSetROISequence"],
    )

    return dcm_structure

@functools.lru_cache()
def get_contours_by_ct_uid_from_structure_uid(structure_set_uid):
    dcm_structure = get_dcm_structure_from_uid(structure_set_uid)

    number_to_name_map = {
        roi_sequence_item.ROINumber: names_map[roi_sequence_item.ROIName]
        for roi_sequence_item in dcm_structure.StructureSetROISequence
        if names_map[roi_sequence_item.ROIName] is not None
    }

    contours_by_ct_uid = pipeline.get_contours_by_ct_uid(dcm_structure, number_to_name_map)

    return contours_by_ct_uid

In [ ]:
structure_uid = ct_uid_to_structure_uid[ct_uid]

In [ ]:
ct_path = pipeline.download_uid(data_path_root, ct_uid, uid_to_url, hash_path)
structure_path = pipeline.download_uid(data_path_root, structure_uid, uid_to_url, hash_path)

In [ ]:
dcm_ct = get_dcm_ct_from_uid(ct_uid)
dcm_structure = get_dcm_structure_from_uid(structure_uid)

In [ ]:
grid_x, grid_y, ct_img = pipeline.create_input_ct_image(dcm_ct)

In [ ]:
contours_by_ct_uid = get_contours_by_ct_uid_from_structure_uid(
    structure_uid
)

In [ ]:
# contours_by_ct_uid

In [ ]:
expansion = 4

_, _, ct_size = mask.get_grid(dcm_ct)
ct_size = tuple(np.array(ct_size) * expansion)
ct_size

In [ ]:
contours_on_this_slice = contours_by_ct_uid[ct_uid].keys()
contours_on_this_slice

In [ ]:
masks = dict()

for structure in contours_on_this_slice:
    if structure in contours_on_this_slice:
        masks[structure] = mask.calculate_expanded_mask(
            contours_by_ct_uid[ct_uid][structure],
            dcm_ct, expansion
        )
    else:
        masks[structure] = np.zeros(ct_size).astype(bool)

In [ ]:
plt.pcolormesh(masks['patient'])

In [ ]:
plt.pcolormesh(masks['brain'])

In [ ]:
def is_mask_a_subset(subset, superset):
    return np.all(np.logical_and(subset, superset) == subset)

In [ ]:
def cmp(x, y):
    mask_x = masks[x]
    mask_y = masks[y]
    
    if is_mask_a_subset(mask_x, mask_y):
        return -1
    if is_mask_a_subset(mask_y, mask_x):
        return 1
    
    disjoint = np.logical_xor(mask_x, mask_y) == np.logical_or(mask_x, mask_y)
    
    if np.any(np.invert(disjoint)):
        raise ValueError(f"Masks ({x}, {y}) are disjoint")

    
    return 0

sorting_key = functools.cmp_to_key(cmp)

sorted(list(contours_on_this_slice), key=sorting_key)

In [ ]:
names = list(set(names_map.values()))
names.remove(None)

names.sort()

def get_ids(structure_name):
    index = names.index(structure_name)
    instance_id = index + 1
    category_id = instance_id + 1
    
    return instance_id, category_id

In [ ]:
for i, name in enumerate(names):
    print(i, name)

In [ ]:
dcm_ct

In [ ]:
HU = dcm_ct.pixel_array * dcm_ct.RescaleSlope + dcm_ct.RescaleIntercept

In [ ]:
np.min(HU)

In [ ]:
np.max(HU)

In [ ]:
2048

In [ ]:
expanded = skimage.transform.resize(HU, ct_size, mode='edge', preserve_range=True, anti_aliasing=False)

In [ ]:
uint16_image = ((expanded + 1024) / 4096 * 2**16).astype(np.uint16)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(uint16_image)
plt.axis('equal')
plt.colorbar()

In [ ]:
png_file = io.BytesIO()

imageio.imsave(png_file, uint16_image, format='png', prefer_uint8=False)

In [ ]:
# Set up the client
client = segments.SegmentsClient(segments_api_key)
dataset_name = 'SimonBiggs/AnimalContours' # Name of a dataset you've created on Segments.ai

sample_name = f"{ct_uid}.png"

asset = client.upload_asset(png_file, filename=sample_name)
image_url = asset["url"]



In [ ]:
# sample_name = f"{ct_uid}.png"

In [ ]:
image_url

In [ ]:
attributes = {
    "image": { "url": image_url }
}
result = client.add_sample(dataset_name, sample_name, attributes)

In [ ]:
result

In [ ]:
expanded

In [ ]:
dcm_ct.pixel_array

In [ ]:
dcm_ct.pixel_array

In [ ]:
asset = client.upload_asset(file, filename="violet.jpg")
image_url = asset["url"]

In [ ]:


# Upload the images to Segments.ai
for i, url in enumerate(image_urls):
    sample_name = 'image_{:05}.jpg'.format(i)
    attributes = {
        "image": { "url": url }
    }
    result = client.add_sample(dataset_name, sample_name, attributes)

In [ ]:
GH_API = "https://api.github.com"
GH_REPO = f"{GH_API}/repos/pymedphys/data"
GH_TAGS = f"{GH_API}/releases/tags/segments.ai"
AUTH = f"Authorization: token {github_api_key}"

In [ ]:
# repo = sys.argv[1]
# tag = sys.argv[2]
# upload_file = sys.argv[3]

# token = os.environ['GITHUB_TOKEN']

# url_template = 'https://{}.github.com/repos/' + repo + '/releases'

# # Create.
# _json = json.loads(urlopen(Request(
#     url_template.format('api'),
#     json.dumps({
#         'tag_name': tag,
#         'name': tag,
#         'prerelease': True,
#     }).encode(),
#     headers={
#         'Accept': 'application/vnd.github.v3+json',
#         'Authorization': 'token ' + token,
#     },
# )).read().decode())
# release_id = _json['id']

# # Upload.
# with open(upload_file, 'br') as myfile:
#     content = myfile.read()
# _json = json.loads(urlopen(Request(
#     url_template.format('uploads') + '/' + str(release_id) + '/assets?' \
#       + urlencode({'name': os.path.split(upload_file)[1]}),
#     content,
#     headers={
#         'Accept': 'application/vnd.github.v3+json',
#         'Authorization': 'token ' + token,
#         'Content-Type': 'application/zip',
#     },
# )).read().decode())

In [ ]:
# masks['bite_block'][:, 1000:1010] = True

# plt.pcolormesh(masks['bite_block'])

In [ ]:
mask_a = masks['eye_left']
mask_b = masks['brain']

disjoint = np.logical_xor(mask_a, mask_b) == np.logical_or(mask_a, mask_b)
np.any(np.invert(disjoint))
# plt.pcolormesh()

In [ ]:
for contour_labels in itertools.permutations(contours_on_this_slice, 2):
    print(contour_labels)
    
    mask_a = masks[contour_labels[0]]
    mask_b = masks[contour_labels[1]]
    
    disjoint = np.logical_xor(
        np.logical_xor(mask_a, mask_b),
        np.logical_or(mask_a, mask_b)
    )
    
    print(np.any(disjoint))

In [ ]:
np.all(np.logical_and(masks['brain'], masks['patient']) == masks['brain'])

In [ ]:
masks